# Prerequisite

In [1]:
import pandas as pd
import requests
import json
import folium
import numpy as np
import time
from scipy.spatial.distance import cdist
from IPython.display import IFrame

# Load datasets into dataframe

In [2]:
employee_addresses = pd.read_csv('data/Employee_Addresses.csv')
bus_stops = pd.read_csv('data/Bus_Stops.csv')

# First things first, getting sense of data

In [3]:
employee_addresses.head()

,address,employee_id
0,"98 Edinburgh St, San Francisco, CA 94112, USA",206
1,"237 Accacia St, Daly City, CA 94014, USA",2081
2,"1835 Folsom St, San Francisco, CA 94103, USA",178
3,"170 Cambridge St, San Francisco, CA 94134, USA",50
4,"16 Roanoke St, San Francisco, CA 94131, USA",1863


In [4]:
bus_stops.head()

,Street_One,Street_Two
0,MISSION ST,ITALY AVE
1,MISSION ST,NEW MONTGOMERY ST
2,MISSION ST,01ST ST
3,MISSION ST,20TH ST
4,MISSION ST,FREMONT ST


In [5]:
print(employee_addresses.count())

address        2191
employee_id    2191
dtype: int64


In [6]:
print(bus_stops.count())

Street_One    119
Street_Two    119
dtype: int64


# Data preprocessing

Bus stops data need to be preprocessed to get valid intersection address for OpenStreetMap API. Street junction/intersection doesn't work well with OpenStreet API. So, some hacks are needed to get it to working to a reasonable level, e.g. swapping street names before and after. Google API should be used for better geocoding.

In [7]:
bus_stops['address'] = bus_stops['Street_One'].astype(str) + ' and ' + bus_stops['Street_Two'].astype(str) + ' , San Francisco, CA'
bus_stops['address2'] = bus_stops['Street_Two'].astype(str) + ' and ' + bus_stops['Street_One'].astype(str) + ', San Francisco, CA'
bus_stops['address3'] = bus_stops['Street_Two'].astype(str) + ' , San Francisco, CA'

# Geocoding from openstreet map API through requests

REQUESTS call (REST API) is used to get the geocoding here. However, there are numerous libraries (e.g. geopy) that can be tested to speed-up the process.

In [8]:
# Bus stops
lat = []
lon = []

#bus_stops = bus_stops[0:20] # Debugging

for index,row in bus_stops.iterrows():
    location_string = bus_stops["address"][index]
    geocode = requests.get('https://nominatim.openstreetmap.org/search.php?q=' + location_string + '&&format=json&&email=itanvir@hotmail.com')
    geocode = json.loads(geocode.text)
    if (len(geocode)<1) | (len(geocode)>8):
        location_string = bus_stops["address2"][index]
        if location_string[0] == '0':
            location_string = location_string[1::]
        geocode = requests.get('https://nominatim.openstreetmap.org/search.php?q=' + location_string + '&&format=json&&email=itanvir@hotmail.com')
        geocode = json.loads(geocode.text)
        if (len(geocode)<1) | (len(geocode)>8):
            location_string = bus_stops["address3"][index]
            if location_string[0] == '0':
                location_string = location_string[1::]
            geocode = requests.get('https://nominatim.openstreetmap.org/search.php?q=' + location_string + '&&format=json&&email=itanvir@hotmail.com')
            geocode = json.loads(geocode.text)
            if len(geocode)<1: # Check if API returns none
                lon.append(lon[-1])
                lat.append(lat[-1])
                continue
    
    #print (index, len(geocode))
    lon.append(float(geocode[0]['lon']))
    lat.append(float(geocode[0]['lat']))
    
bus_stops['longitude'] = np.array(lon)
bus_stops['latitude'] = np.array(lat)

In [9]:
bus_stops.head()

,Street_One,Street_Two,address,address2,address3,longitude,latitude
0,MISSION ST,ITALY AVE,"MISSION ST and ITALY AVE , San Francisco, CA","ITALY AVE and MISSION ST, San Francisco, CA","ITALY AVE , San Francisco, CA",-122.439211,37.718696
1,MISSION ST,NEW MONTGOMERY ST,"MISSION ST and NEW MONTGOMERY ST , San Francis...","NEW MONTGOMERY ST and MISSION ST, San Francisc...","NEW MONTGOMERY ST , San Francisco, CA",-122.400821,37.787704
2,MISSION ST,01ST ST,"MISSION ST and 01ST ST , San Francisco, CA","01ST ST and MISSION ST, San Francisco, CA","01ST ST , San Francisco, CA",-122.397352,37.789470
3,MISSION ST,20TH ST,"MISSION ST and 20TH ST , San Francisco, CA","20TH ST and MISSION ST, San Francisco, CA","20TH ST , San Francisco, CA",-122.418988,37.758812
4,MISSION ST,FREMONT ST,"MISSION ST and FREMONT ST , San Francisco, CA","FREMONT ST and MISSION ST, San Francisco, CA","FREMONT ST , San Francisco, CA",-122.396762,37.790306


In [10]:
# Employee address
lat = []
lon = []

#employee_addresses = employee_addresses[0:100] # Debugging
for index,row in employee_addresses.iterrows():
    location_string = employee_addresses["address"][index]
    geocode = requests.get('https://nominatim.openstreetmap.org/search.php?q=' + location_string + '&&format=json&&email=itanvir@hotmail.com')
    geocode = json.loads(geocode.text)
    if len(geocode)<1:
        lon.append(lon[-1])
        lat.append(lat[-1])
        continue
    lon.append(float(geocode[0]['lon']))
    lat.append(float(geocode[0]['lat']))
    #print (index, len(geocode))
    time.sleep(1) # Delay 1 second to meet OSM usage policy, 1 request per second max

employee_addresses['longitude'] = np.array(lon)
employee_addresses['latitude'] = np.array(lat)

In [11]:
geocode

[{'place_id': 205942459,
  'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
  'osm_type': 'way',
  'osm_id': 25821954,
  'boundingbox': ['37.762212', '37.762312', '-122.413948', '-122.413848'],
  'lat': '37.762262',
  'lon': '-122.413898',
  'display_name': '499, Treat Avenue, Mission District, San Francisco, San Francisco City and County, California, 94110, United States',
  'class': 'place',
  'type': 'house',
  'importance': 0.721}]

# Folium map

We have geocded and received our coordinates. Let's check out how the points look in the map.

In [12]:
m = folium.Map(location=[37.75, -122.4], zoom_start=12)

# Bus stops markers
for i in range(0,len(bus_stops)):
    folium.Marker([bus_stops.iloc[i]['latitude'], bus_stops.iloc[i]['longitude']]).add_to(m)


# Employee addresses markers
for i in range(0,len(employee_addresses)):
    folium.Marker([employee_addresses.iloc[i]['latitude'], employee_addresses.iloc[i]['longitude']], icon = folium.Icon(color='red')).add_to(m)
 
m.save('index.html')
display(m)

In [2]:
IFrame("index.html", width=1000, height=600)

# Key optimization metric

In reality, we should use a metric related to walk score or walk distance. Google Map API can provide that. As an alternative, we are using Manhattan distance for now.

In [13]:
A = np.column_stack([bus_stops['longitude'], bus_stops['latitude']])
B = np.column_stack([employee_addresses['longitude'], employee_addresses['latitude']])
distance = cdist(A, B, metric='cityblock')
distance_sum = np.sum(distance, 1)

# Best 10 stops

The stops sorted by least sum distances are considered as the best 10 stops.

In [21]:
distance_sort = np.sort(distance_sum)
index_10 = np.argsort(distance_sum)[0:10]

In [22]:
bus_stops.iloc[index_10, :]

,Street_One,Street_Two,address,address2,address3,longitude,latitude
63,MISSION ST,RICHLAND AVE,"MISSION ST and RICHLAND AVE , San Francisco, CA","RICHLAND AVE and MISSION ST, San Francisco, CA","RICHLAND AVE , San Francisco, CA",-122.424427,37.735869
22,MISSION ST,PARK ST,"MISSION ST and PARK ST , San Francisco, CA","PARK ST and MISSION ST, San Francisco, CA","PARK ST , San Francisco, CA",-122.424413,37.736640
9,MISSION ST,SAINT MARYS AVE,"MISSION ST and SAINT MARYS AVE , San Francisco...","SAINT MARYS AVE and MISSION ST, San Francisco, CA","SAINT MARYS AVE , San Francisco, CA",-122.427413,37.734522
81,MISSION ST,COLLEGE TER,"MISSION ST and COLLEGE TER , San Francisco, CA","COLLEGE TER and MISSION ST, San Francisco, CA","COLLEGE TER , San Francisco, CA",-122.425904,37.734351
68,MISSION ST,HIGHLAND AVE,"MISSION ST and HIGHLAND AVE , San Francisco, CA","HIGHLAND AVE and MISSION ST, San Francisco, CA","HIGHLAND AVE , San Francisco, CA",-122.423950,37.737629
106,MISSION ST,BOSWORTH ST,"MISSION ST and BOSWORTH ST , San Francisco, CA","BOSWORTH ST and MISSION ST, San Francisco, CA","BOSWORTH ST , San Francisco, CA",-122.426809,37.733437
78,MISSION ST,COLLEGE AVE,"MISSION ST and COLLEGE AVE , San Francisco, CA","COLLEGE AVE and MISSION ST, San Francisco, CA","COLLEGE AVE , San Francisco, CA",-122.423636,37.733591
45,MISSION ST,APPLETON AVE,"MISSION ST and APPLETON AVE , San Francisco, CA","APPLETON AVE and MISSION ST, San Francisco, CA","APPLETON AVE , San Francisco, CA",-122.423838,37.739071
95,MISSION ST,RANDALL ST,"MISSION ST and RANDALL ST , San Francisco, CA","RANDALL ST and MISSION ST, San Francisco, CA","RANDALL ST , San Francisco, CA",-122.423889,37.739814
111,MISSION ST,BROOK ST,"MISSION ST and BROOK ST , San Francisco, CA","BROOK ST and MISSION ST, San Francisco, CA","BROOK ST , San Francisco, CA",-122.423451,37.740685


# Folium map for best 10 stops

In [35]:
m = folium.Map(location=[37.75, -122.4], zoom_start=13)

# Bus stops markers
for i in range(10):
    folium.Marker([bus_stops.iloc[index_10[i]]['latitude'], bus_stops.iloc[index_10[i]]['longitude']]).add_to(m)
    
m.save('results.html')
display(m)

In [1]:
IFrame("results.html", width=1000, height=600)

# Caveats

Openstreetmap API lacks geocoding accuracy. Google Map API should improve the optimization results.